In [7]:
import os
import pickle
from collections import defaultdict
from itertools import combinations

In [2]:
file_path = "../../output/run_output_final/final_diarization_output_MY_DATASET_ALL.pckl"

In [3]:
dia_output = pickle.load(open(file_path, "rb"))

In [9]:
dia_output.keys()

dict_keys(['StarTalk_Cosmic_780_850', 'StarTalk_Mars_1680_1780', 'StarTalk_Mars_1109_1175', 'StarTalk_CMBR_270_308', 'StarTalk_Farming_2405_2500', 'StarTalk_Questions_1250_1350', 'StarTalk_FlyingVehicles_2446_2508', 'StarTalk_Mars_380_450', 'StarTalk_Mars_1345_1426', 'StarTalk_Consciousness_2314_2387', 'StarTalk_Sleep_1602_1639', 'StarTalk_Questions_690_750', 'StarTalk_Farming_1059_1180', 'StarTalk_Sleep_382_450', 'StarTalk_Mars_927_1025', 'StarTalk_FlyingVehicles_780_811', 'StarTalk_FlyingVehicles_2670_2710', 'StarTalk_Consciousness_1075_1175', 'StarTalk_Cosmic_1550_1620', 'StarTalk_Sleep_748_796', 'StarTalk_Mars_2210_2270', 'StarTalk_Mars_2315_2376', 'StarTalk_FlyingVehicles_674_719', 'StarTalk_Consciousness_2190_2254', 'StarTalk_Sleep_2470_2551', 'StarTalk_Sleep_2099_2160', 'StarTalk_Cosmic_1820_1900', 'MagnusCarlson_542_599', 'StarTalk_Mars_2020_2095', 'StarTalk_Consciousness_683_784', 'StarTalk_Cosmic_1350_1442', 'StarTalk_CMBR_190_225', 'StarTalk_FlyingVehicles_949_1000', 'StarTa

In [5]:
dia_output[list(dia_output.keys())[0]]["audio_12"]

{'SPEAKER_00': [(0.01, 6.99),
  (7.9, 8.01),
  (8.36, 8.41),
  (16.95, 17.55),
  (17.85, 21.5),
  (23.13, 23.73),
  (31.66, 38.29),
  (39.84, 42.89),
  (43.06, 43.85),
  (44.86, 45.0),
  (45.02, 45.49),
  (46.09, 46.1)],
 'SPEAKER_02': [(7.89, 7.9),
  (8.01, 8.36),
  (8.41, 12.23),
  (13.08, 14.27),
  (14.78, 17.28),
  (17.89, 18.58),
  (21.81, 21.83),
  (45.49, 45.59),
  (45.88, 46.09),
  (47.0, 47.33),
  (50.23, 54.51),
  (54.85, 70.09)],
 'SPEAKER_01': [(18.58, 18.6),
  (21.83, 23.13),
  (24.03, 30.57),
  (31.1, 32.42),
  (35.53, 39.87),
  (40.89, 41.26),
  (45.0, 51.98),
  (52.47, 53.37)]}

In [15]:
def calculate_total_duration(speakers_intervals):
    total_duration = 0
    for intervals in speakers_intervals.values():
        for start, end in intervals:
            total_duration += (end - start)
    return total_duration

def calculate_overlap(speakers_intervals):
    # Convert intervals to events (start and end points)
    events = []
    for speaker, intervals in speakers_intervals.items():
        for start, end in intervals:
            events.append((start, 'start', speaker))
            events.append((end, 'end', speaker))
    
    # Sort events by time
    events.sort()

    # Dictionary to keep track of active speakers and total overlap time
    active_speakers = defaultdict(int)
    total_overlap = 0
    last_time = 0

    # Process events
    for time, event_type, speaker in events:
        if event_type == 'start':
            if len(active_speakers) > 1:
                total_overlap += (time - last_time)
            active_speakers[speaker] += 1
        elif event_type == 'end':
            if len(active_speakers) > 1:
                total_overlap += (time - last_time)
            active_speakers[speaker] -= 1
            if active_speakers[speaker] == 0:
                del active_speakers[speaker]
        
        last_time = time
        
    total_length = calculate_total_duration(speakers_intervals)

    return total_overlap/total_length

In [16]:
overlap = {}
for video_file in dia_output.keys():
    overlap[video_file] = calculate_overlap(dia_output[video_file]["audio_12"])

In [17]:
overlap

{'StarTalk_Cosmic_780_850': 0.11863024867509157,
 'StarTalk_Mars_1680_1780': 0.12220593624038105,
 'StarTalk_Mars_1109_1175': 0.08670172802404198,
 'StarTalk_CMBR_270_308': 0.03562340966921114,
 'StarTalk_Farming_2405_2500': 0.04453395994181483,
 'StarTalk_Questions_1250_1350': 0.04799338022341754,
 'StarTalk_FlyingVehicles_2446_2508': 0.055820846369175,
 'StarTalk_Mars_380_450': 0.09417409184372849,
 'StarTalk_Mars_1345_1426': 0.11093463653023831,
 'StarTalk_Consciousness_2314_2387': 0.02826698238863399,
 'StarTalk_Sleep_1602_1639': 0.04590256265840616,
 'StarTalk_Questions_690_750': 0.16734296380457714,
 'StarTalk_Farming_1059_1180': 0.01607436816113106,
 'StarTalk_Sleep_382_450': 0.09543065103113635,
 'StarTalk_Mars_927_1025': 0.130001896453632,
 'StarTalk_FlyingVehicles_780_811': 0.15095936794582407,
 'StarTalk_FlyingVehicles_2670_2710': 0.13578345070422537,
 'StarTalk_Consciousness_1075_1175': 0.0870590648072187,
 'StarTalk_Cosmic_1550_1620': 0.13298983428491853,
 'StarTalk_Sleep_